In [1]:
# Provides ways to work with large multidimensional arrays
import numpy as np 
# Allows for further data manipulation and analysis
import pandas as pd

# In Anaconda -> Environments -> Not Installed -> pandas-datareader -> Apply
from pandas_datareader import data as web # Reads stock data 
import matplotlib.pyplot as plt # Plotting
import matplotlib.dates as mdates # Styling dates
%matplotlib inline

import cufflinks as cf
import plotly.express as px
import plotly.graph_objects as go

# Make Plotly work in your Jupyter Notebook
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
# Use Plotly locally
cf.go_offline()

import warnings
warnings.simplefilter("ignore")

# pip install yfinance in Qt Console or Powershell Prompt on Windows
import yfinance as yf

from plotly.subplots import make_subplots

# Install the Python Technical Analysis library
# pip install ta in Qt Console or Powershell prompt
from ta.trend import MACD
from ta.momentum import StochasticOscillator

# TA-Lib is used by traders to perform technical analysis
# Go here : https://github.com/mrjbq7/ta-lib#dependencies
import talib as ta

ModuleNotFoundError: No module named 'talib'

## Ichimoku Kinko Hyo

The Ichimoku (One Look) is considered an all in one indicator. It provides information on momentum, support and resistance. It is made up of 5 lines. If you are a short term trader you create 1 minute or 6 hour. Long term traders focus on day or weekly data.

 - Conversion Line (Tenkan-sen) : Represents support, resistance and reversals. Used to measure short term trends.
 - Baseline (Kijun-sen) : Represents support, resistance and confirms trend changes. Allows you to evaluate the strength of medium term trends. Called the baseline because it lags the price.
 - Leading Span A (Senkou A) : Used to identify future areas of support and resistance
 - Leading Span B (Senkou B) : Other line used to identify suture support and resistance
 - Lagging Span (Chikou) : Shows possible support and resistance. It is used to confirm signals obtained from other lines.
 - Cloud (Kumo) : Space between Span A and B. Represents the divergence in price evolution.
 
Formulas

 - Lagging Span = Price shifted back 26 periods
 - Base Line = (Highest Value in period + Lowest value in period)/2 (26 Sessions)
 - Conversion Line = (Highest Value in period + Lowest value in period)/2 (9 Sessions)
 - Leading Span A = (Conversion Value + Base Value)/2 (26 Sessions)
 - Leading Span B = (Conversion Value + Base Value)/2 (52 Sessions)

## Download Stock Data

 - Valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
 - Valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo

In [2]:
amd_df = yf.download(tickers="AMD", period="1y", interval='1d')

[*********************100%***********************]  1 of 1 completed


## Conversion Line (Tenkan-sen)

In [3]:
# Conversion Line = (Highest Value in period + Lowest value in period)/2 (9 Sessions)

hi_val = amd_df['High'].rolling(window=9).max()
low_val = amd_df['Low'].rolling(window=9).min()
amd_df['Conversion'] = (hi_val + low_val) / 2

## Baseline (Kijun-sen)

In [4]:
# Base Line = (Highest Value in period + Lowest value in period)/2 (26 Sessions)
hi_val2 = amd_df['High'].rolling(window=26).max()
low_val2 = amd_df['Low'].rolling(window=26).min()
amd_df['Baseline'] = (hi_val2 + low_val2) / 2

## Leading Span A (Senkou A)

In [5]:
# Leading Span A = (Conversion Value + Base Value)/2 (26 Sessions)
amd_df['SpanA'] = ((amd_df['Conversion'] + amd_df['Baseline']) / 2).shift(26)

## Leading Span B (Senkou B)

In [6]:
# Leading Span B = (Conversion Value + Base Value)/2 (52 Sessions)
# You need the conversion and baseline using 52 sessions
hi_val3 = amd_df['High'].rolling(window=52).max()
low_val3 = amd_df['Low'].rolling(window=52).min()
amd_df['SpanB'] = ((hi_val3 + low_val3) / 2).shift(26)

## Lagging Span (Chikou)

In [7]:
# Lagging Span = Price shifted back 26 periods
amd_df['Lagging'] = amd_df['Close'].shift(-26)

## Candlesticks

In [8]:
candle = go.Candlestick(x=amd_df.index, open=amd_df['Open'],
                       high=amd_df['High'], low=amd_df['Low'],
                       close=amd_df['Close'], name='Candlestick')

In [9]:
# Sets fill color to green when value greater or equal to 1
# and red otherwise
def get_fill_color(label):
    if label >= 1:
        return 'rgba(0,250,0,0.4)'
    else:
        return 'rgba(250,0,0,0.4)'

## Plot

In [10]:
# We will use 2 dataframes
# 1 will contain the fill data between the spans
# The other will be stored with the original data in df1
df1 = amd_df.copy()

fig = go.Figure()

# Where SpanA is greater than SpanB give label a value of 1 or 0 if not
amd_df['label'] = np.where(amd_df['SpanA'] > amd_df['SpanB'], 1, 0)

# Shift 1 period, compare dataframe for inequality with the cumulative 
# sum and store in group
amd_df['group'] = amd_df['label'].ne(amd_df['label'].shift()).cumsum()

# Get a groupby object that contains information on the group
amd_df = amd_df.groupby('group')

# Cycle through the data pertaining to the fill between spans
dfs = []
for name, data in amd_df:
    dfs.append(data)

    
# Add 2 traces to the fig object for each time the spans cross
# and then define the fill using fill='tonexty' for the second trace
for df in dfs:
    fig.add_traces(go.Scatter(x=df.index, y = df.SpanA,
                              line = dict(color='rgba(0,0,0,0)')))
    
    fig.add_traces(go.Scatter(x=df.index, y = df.SpanB,
                              line = dict(color='rgba(0,0,0,0)'),
                              fill='tonexty', 
                              fillcolor = get_fill_color(df['label'].iloc[0])))


# Create plots for all of the nonfill data
baseline = go.Scatter(x=df1.index, y=df1['Baseline'], 
                   line=dict(color='pink', width=2), name="Baseline")

conversion = go.Scatter(x=df1.index, y=df1['Conversion'], 
                  line=dict(color='black', width=1), name="Conversion")

lagging = go.Scatter(x=df1.index, y=df1['Lagging'], 
                  line=dict(color='purple', width=2, dash='dot'), name="Lagging")

span_a = go.Scatter(x=df1.index, y=df1['SpanA'],
                  line=dict(color='green', width=2, dash='dot'), name="Span A")

span_b = go.Scatter(x=df1.index, y=df1['SpanB'],
                    line=dict(color='red', width=1, dash='dot'), name="Span B")

# Add plots to the figure
fig.add_trace(candle)
fig.add_trace(baseline)
fig.add_trace(conversion)
fig.add_trace(lagging)
fig.add_trace(span_a)
fig.add_trace(span_b)

fig.update_layout(height=900, width=1000, 
                  showlegend=True)

fig.show()

## What the Lines Mean

 - Lagging Span : When above the price it is bullish and when below bearish. It is used with other indicators because it is mainly a filter.
 - Baseline : When below price this is considered support. When above price this is considered resistance. We are in an uptrend when the slope increases and vice versa. The slope of the curve tells us the strength of the trend.
 - Conversion : We focus on its position versus the Baseline. When the Conversion crosses above the Baseline we are in an upward trend and vice versa. This is considered a strong indicator when above the Cloud and weak when below.
 - Cloud : The thicker the Cloud, the stronger the trend and vice versa. When the Spans cross many times we are in a range. When they cross this is a sign of a reversal of trend. 
 
1. Lagging span is bullish AMD till October 27th which is our last reading. Trending down though.
2. The baseline is showing a strong support and has a great upward slope
3. The Conversion is above the Baseline and the cloud which are great bullish trends
4. On November 25th the Cloud crossed into the red signaling a reversal in trend (Bearish) but then turned bullish again November 26th

## Get Ichimoku Function

In [11]:
def get_Ichimoku(ticker):
    df = yf.download(tickers=ticker, period="1y", interval='1d')
    
    # Conversion
    hi_val = df['High'].rolling(window=9).max()
    low_val = df['Low'].rolling(window=9).min()
    df['Conversion'] = (hi_val + low_val) / 2
    
    # Baseline
    hi_val2 = df['High'].rolling(window=26).max()
    low_val2 = df['Low'].rolling(window=26).min()
    df['Baseline'] = (hi_val2 + low_val2) / 2
    
    # Spans
    df['SpanA'] = ((df['Conversion'] + df['Baseline']) / 2).shift(26)
    hi_val3 = df['High'].rolling(window=52).max()
    low_val3 = df['Low'].rolling(window=52).min()
    df['SpanB'] = ((hi_val3 + low_val3) / 2).shift(26)
    df['Lagging'] = df['Close'].shift(-26)
    
    candle = go.Candlestick(x=df.index, open=df['Open'],
                       high=df['High'], low=df['Low'],
                       close=df['Close'], name='Candlestick')
    
    df1 = df.copy()
    fig = go.Figure()
    df['label'] = np.where(df['SpanA'] > df['SpanB'], 1, 0)

    df['group'] = df['label'].ne(df['label'].shift()).cumsum()

    df = df.groupby('group')

    dfs = []
    for name, data in df:
        dfs.append(data)
        
    for df in dfs:
        fig.add_traces(go.Scatter(x=df.index, y = df.SpanA,
                              line = dict(color='rgba(0,0,0,0)')))
    
        fig.add_traces(go.Scatter(x=df.index, y = df.SpanB,
                              line = dict(color='rgba(0,0,0,0)'),
                              fill='tonexty', 
                              fillcolor = get_fill_color(df['label'].iloc[0])))
    
    baseline = go.Scatter(x=df1.index, y=df1['Baseline'], 
                   line=dict(color='pink', width=2), name="Baseline")

    conversion = go.Scatter(x=df1.index, y=df1['Conversion'], 
                  line=dict(color='black', width=1), name="Conversion")

    lagging = go.Scatter(x=df1.index, y=df1['Lagging'], 
                  line=dict(color='purple', width=2, dash='dot'), name="Lagging")

    span_a = go.Scatter(x=df1.index, y=df1['SpanA'],
                  line=dict(color='green', width=2, dash='dot'), name="Span A")

    span_b = go.Scatter(x=df1.index, y=df1['SpanB'],
                    line=dict(color='red', width=1, dash='dot'), name="Span B")

    # Add plots to the figure
    fig.add_trace(candle)
    fig.add_trace(baseline)
    fig.add_trace(conversion)
    fig.add_trace(lagging)
    fig.add_trace(span_a)
    fig.add_trace(span_b)

    fig.update_layout(height=900, width=1000, 
                  showlegend=True)

    fig.show()

In [12]:
get_Ichimoku("AMZN")

[*********************100%***********************]  1 of 1 completed
